In [ ]:
import pandas as pd

path = "data/all_apps_wide_2026-01-28.csv"

df = pd.read_csv(
    path,
    sep=",",
)


#1. participant._current_page_name == results -> remove non experiment rows
mask = df['participant._current_page_name'] == 'Results'
df = df[mask]

#2. create dict of column names to access them easier
cols = {
    'comprehension_attempts': 'intro.1.player.comprehension_attempts',
    'choice': 'intro.1.group.retailer_choice',
    'payoff': 'experiment.1.player.payoff',
    'profit': 'experiment.1.player.profit'
}

#3. create column for total mistakes during comprehension check phase

def mistakes_per_comprehension_row(row):
    values = row.split(',')
    clean_values = [int(x.split(':')[1].strip().strip('}')) for x in values]
    corrected_values = [x-1 for x in clean_values]

    return sum(corrected_values)

df['defined.comprehension_mistakes'] = df[cols['comprehension_attempts']].apply(mistakes_per_comprehension_row)
cols['comprehension_mistakes'] = 'defined.comprehension_mistakes'

#4. ensure that flags differ: 1/0 or 0/1

for index, row in df.iterrows():
    if row['session.config.force_retailer_first'] == row['session.config.force_supplier_first']:
        raise Exception('session.config.force_retailer_first = session.config.force_supplier_first | Random Assignment')
    
